<a href="https://colab.research.google.com/github/Anurag943/Advanced-Ocular-Disease-Recognition/blob/main/Advanced_Ocular_Disease_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install p7zip-full
!7z x '/content/drive/MyDrive/Ocular Disease Recognition/Test_Set.zip'


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Ocular Disease Recognition/                                                            1 file, 1742451361 bytes (1662 MiB)

Extracting archive: /content/drive/MyDrive/Ocular Disease Recognition/Test_Set.zip
--
Path = /content/drive/MyDrive/Ocular Disease Recognition/Test_Set.zip
Type = zip
Physical Size = 1742451361

  0%      1% 18 - Test_Set/Test/112.png 

In [ ]:
!7z x '/content/drive/MyDrive/Ocular Disease Recognition/Train_Set.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Ocular Disease Recognition/                                                            
ERROR: No more files
/content/drive/MyDrive/Ocular Disease Recognition/Train_Set.zip



System ERROR:
Unknown error -2147024872


In [ ]:
!7z x '/content/drive/MyDrive/Ocular Disease Recognition/Training_Set.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Ocular Disease Recognition/                                                            1 file, 4681355361 bytes (4465 MiB)

Extracting archive: /content/drive/MyDrive/Ocular Disease Recognition/Training_Set.zip
--
Path = /content/drive/MyDrive/Ocular Disease Recognition/Training_Set.zip
Type = zip
Physical Size = 4681355361
64-bit = +

  0%      0% 23 - Training_Set/Training/1016.png                                          0% 28 - Training_Set/Training/1020.png                      

In [ ]:
!7z x '/content/drive/MyDrive/Ocular Disease Recognition/Evaluation_Set.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Ocular Disease Recognition/                                                            1 file, 1564446026 bytes (1492 MiB)

Extracting archive: /content/drive/MyDrive/Ocular Disease Recognition/Evaluation_Set.zip
--
Path = /content/drive/MyDrive/Ocular Disease Recognition/Evaluation_Set.zip
Type = zip
Physical Size = 1564446026

  0%      2% 22 - Evaluation_Set/Validation/116.png                                             2% 23         3% 33 - Evaluation_Set/Validation/126.png

In [ ]:
%%writefile model.py

import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.data.experimental import AUTOTUNE

def create_model(epoch=10, learning_rate=0.01, regularizer,dropout_rate=0.5):
    # Load InceptionResNetV2 model with pre-trained weights and without the top classification layer
    base_model = InceptionResNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

    # Freeze the base model layers to prevent them from being trained (transfer learning)
    base_model.trainable = False

    # Define the custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),  # This replaces Flatten to reduce overfitting risk
        Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(46, activation='softmax')  # 46 output classes for classification
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    # Define data augmentation for training images
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,  # Normalize pixel values
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Validation data generator with only rescaling
    valid_datagen = ImageDataGenerator(rescale=1.0/255)

    # Example for loading data from directories (adjust paths accordingly)
    train_data = train_datagen.flow_from_directory(
        'path_to_train_data',  # Replace with the actual path
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    valid_data = valid_datagen.flow_from_directory(
        'path_to_validation_data',  # Replace with the actual path
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    # Prefetch to optimize data pipeline (AUTOTUNE adjusts buffer size dynamically)
    train_data = train_data.prefetch(AUTOTUNE)
    valid_data = valid_data.prefetch(AUTOTUNE)

    # Train the model
    history = model.fit(
        train_data,
        validation_data=valid_data,
        epochs=epoch,  # Adjust number of epochs based on your needs
        steps_per_epoch=train_data.samples // train_data.batch_size,
        validation_steps=valid_data.samples // valid_data.batch_size
    )

    # Save the trained model
    model.save('inception_resnet_model.h5')

    return history, model


Overwriting model.py


In [ ]:
%%writefile visualizations.py


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


def plot_history(history):
  history = history.history()
  acc = history['accuracy']
  val_acc = history['val_accuracy']
  loss = history['loss']
  val_loss = history['val_loss']
  epochs = range(1, len(acc)+1)
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()

def plot_confusion_matrix(model, test_data):
  y_true = []
  y_pred = []
  for images, labels in test_data:
    y_true.extend(np.argmax(labels, axis=1))
    y_pred.extend(np.argmax(model.predict(images), axis=1))
    # Create a confusion matrix
  cm = confusion_matrix(y_true, y_pred)
  # Plot the confusion matrix
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_data.class_indices, yticklabels=test_data.class_indices)
  plt.xlabel('Predicted')
  plt.ylabel('True')
  plt.title('Confusion Matrix')
  plt.show()

  # Create a classification report
  report = classification_report(y_true, y_pred, target_names=test_data.class_indices.keys())
  print(report)
  return cm, report



Writing visualizations.py


In [ ]:
%%writefile load_dataset.py


from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data():
    # Define data augmentation for training images
    train_datagen = ImageDataGenerator(
        rescale=1.0/255  # Normalize pixel values
    )

    # Define validation data generator with only rescaling
    valid_datagen = ImageDataGenerator(rescale=1.0/255)

    # Load training data
    train_data = train_datagen.flow_from_directory(
        'path_to_train_data',  # Replace with the actual path
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    # Load validation data
    valid_data = valid_datagen.flow_from_directory(
        'path_to_validation_data',  # Replace with the actual path
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_data, valid_data

Writing load_dataset.py


In [ ]:
%%writefile best_hyperparemetes.py

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from model import create_model

# Define hyperparameters to try
hyperparameters_to_try = [
    {"learning_rate": 0.0001, "dropout_rate": 0.5, "unfreeze_layers": 16,
     "augment_params": {"rotation_range": 40, "width_shift_range": 0.3, "height_shift_range": 0.3,
                        "shear_range": 0.3, "zoom_range": 0.3, "horizontal_flip": True, "fill_mode": 'nearest'}},

    {"learning_rate": 0.0001, "dropout_rate": 0.4, "unfreeze_layers": 16,
     "augment_params": {"rotation_range": 30, "width_shift_range": 0.2, "height_shift_range": 0.2,
                        "shear_range": 0.2, "zoom_range": 0.2, "horizontal_flip": True, "fill_mode": 'nearest'}},

    {"learning_rate": 0.0001, "dropout_rate": 0.3, "unfreeze_layers": 16,
     "augment_params": {"rotation_range": 20, "width_shift_range": 0.1, "height_shift_range": 0.1,
                        "shear_range": 0.1, "zoom_range": 0.1, "horizontal_flip": True, "fill_mode": 'nearest'}},

    {"learning_rate": 0.0001, "dropout_rate": 0.5, "unfreeze_layers": 8,
     "augment_params": {"rotation_range": 40, "width_shift_range": 0.3, "height_shift_range": 0.3,
                        "shear_range": 0.3, "zoom_range": 0.3, "horizontal_flip": True, "fill_mode": 'nearest'}},

    {"learning_rate": 0.00001, "dropout_rate": 0.5, "unfreeze_layers": 16,
     "augment_params": {"rotation_range": 20, "width_shift_range": 0.1, "height_shift_range": 0.1,
                        "shear_range": 0.1, "zoom_range": 0.1, "horizontal_flip": True, "fill_mode": 'nearest'}},

    {"learning_rate": 0.00001, "dropout_rate": 0.4, "unfreeze_layers": 16,
     "augment_params": {"rotation_range": 30, "width_shift_range": 0.2, "height_shift_range": 0.2,
                        "shear_range": 0.2, "zoom_range": 0.2, "horizontal_flip": True, "fill_mode": 'nearest'}},

    {"learning_rate": 0.00001, "dropout_rate": 0.3, "unfreeze_layers": 16,
     "augment_params": {"rotation_range": 40, "width_shift_range": 0.3, "height_shift_range": 0.3,
                        "shear_range": 0.3, "zoom_range": 0.3, "horizontal_flip": True, "fill_mode": 'nearest'}},

    {"learning_rate": 0.00001, "dropout_rate": 0.5, "unfreeze_layers": 8,
     "augment_params": {"rotation_range": 20, "width_shift_range": 0.1, "height_shift_range": 0.1,
                        "shear_range": 0.1, "zoom_range": 0.1, "horizontal_flip": True, "fill_mode": 'nearest'}},
]


def train_and_evaluate(hyperparams, train_data, valid_data):
    # Create model with the specified hyperparameters
    model = create_model(
        learning_rate=hyperparams["learning_rate"],
        dropout_rate=hyperparams["dropout_rate"],
        unfreeze_layers=hyperparams["unfreeze_layers"]
    )

    # Define data augmentation based on hyperparameters
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,  # Normalize pixel values
        rotation_range=hyperparams["augment_params"].get("rotation_range", 0),
        width_shift_range=hyperparams["augment_params"].get("width_shift_range", 0),
        height_shift_range=hyperparams["augment_params"].get("height_shift_range", 0),
        shear_range=hyperparams["augment_params"].get("shear_range", 0),
        zoom_range=hyperparams["augment_params"].get("zoom_range", 0),
        horizontal_flip=hyperparams["augment_params"].get("horizontal_flip", False),
        fill_mode=hyperparams["augment_params"].get("fill_mode", 'nearest')
    )

    # Load data
    train_data = train_datagen.flow_from_directory(
        'path_to_train_data',  # Replace with the actual path
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    valid_datagen = ImageDataGenerator(rescale=1.0/255)
    valid_data = valid_datagen.flow_from_directory(
        'path_to_validation_data',  # Replace with the actual path
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    # Train the model
    history = model.fit(
        train_data,
        validation_data=valid_data,
        epochs=10,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        validation_steps=valid_data.samples // valid_data.batch_size
    )

    # Evaluate performance (e.g., validation accuracy)
    val_accuracy = history.history['val_accuracy'][-1]  # Last epoch's validation accuracy
    return val_accuracy

def find_best_hyperparameters(hyperparameters_to_try, train_data, valid_data):
    best_accuracy = 0
    best_hyperparameters = None

    for params in hyperparameters_to_try:
        print(f"Testing hyperparameters: {params}")
        accuracy = train_and_evaluate(params, train_data, valid_data)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = params

    print(f"Best Hyperparameters: {best_hyperparameters} with accuracy: {best_accuracy}")
    return best_hyperparameters

if __name__ == "__main__":
    # Load data
    train_data, valid_data = load_data()

    # Find the best hyperparameters
    best_params = find_best_hyperparameters(hyperparameters_to_try, train_data, valid_data)

    # Output the best hyperparameters
    print(f"Best Hyperparameters: {best_params}")


Writing best_hyperparemetes.py


In [ ]:
%%writefile best_hyperparemters.py
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from model import create_model, train_model, load_data

# Define hyperparameters to try
hyperparameters_to_try = [
    {"learning_rate": 0.0001, "dropout_rate": 0.5, "unfreeze_layers": 16,
     "augment_params": {"rotation_range": 40, "width_shift_range": 0.3, "height_shift_range": 0.3,
                        "shear_range": 0.3, "zoom_range": 0.3, "horizontal_flip": True, "fill_mode": 'nearest'}},

    # Add more hyperparameters to try...
]

def train_and_evaluate(hyperparams, data_path, batch_size, epochs):
    train_data, valid_data = load_data(data_path, batch_size)

    # Apply data augmentation
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=hyperparams["augment_params"].get("rotation_range", 0),
        width_shift_range=hyperparams["augment_params"].get("width_shift_range", 0),
        height_shift_range=hyperparams["augment_params"].get("height_shift_range", 0),
        shear_range=hyperparams["augment_params"].get("shear_range", 0),
        zoom_range=hyperparams["augment_params"].get("zoom_range", 0),
        horizontal_flip=hyperparams["augment_params"].get("horizontal_flip", False),
        fill_mode=hyperparams["augment_params"].get("fill_mode", 'nearest')
    )

    train_data = train_datagen.flow_from_directory(
        data_path + '/train',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Create model with the specified hyperparameters
    model = create_model(
        learning_rate=hyperparams["learning_rate"],
        dropout_rate=hyperparams["dropout_rate"],
        unfreeze_layers=hyperparams["unfreeze_layers"]
    )

    # Train the model
    history = train_model(model, train_data, valid_data, epochs)

    # Evaluate performance (e.g., validation accuracy)
    val_accuracy = history.history['val_accuracy'][-1]  # Last epoch's validation accuracy
    return val_accuracy


def find_best_hyperparameters(hyperparameters_to_try, data_path, batch_size, epochs):
    best_accuracy = 0
    best_hyperparameters = None

    for params in hyperparameters_to_try:
        print(f"Testing hyperparameters: {params}")
        accuracy = train_and_evaluate(params, data_path, batch_size, epochs)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameters = params

    print(f"Best Hyperparameters: {best_hyperparameters} with accuracy: {best_accuracy}")
    return best_hyperparameters


if __name__ == "__main__":
    data_path = 'path_to_data'
    batch_size = 32
    epochs = 10

    # Find the best hyperparameters
    best_params = find_best_hyperparameters(hyperparameters_to_try, data_path, batch_size, epochs)

    # Train the model with the best hyperparameters
    import model
    model.main(data_path, batch_size, epochs, best_params["learning_rate"], best_params["dropout_rate"], best_params["unfreeze_layers"])

In [ ]:
%%writefile model.py

import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.data.experimental import AUTOTUNE

def create_model(learning_rate, dropout_rate, unfreeze_layers):
    # Load InceptionResNetV2 model with pre-trained weights and without the top classification layer
    base_model = InceptionResNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

    # Unfreeze specified layers for training
    for layer in base_model.layers[-unfreeze_layers:]:
        layer.trainable = True

    # Define the custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(dropout_rate),
        Dense(46, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def train_model(model, train_data, valid_data, epochs):
    # Prefetch to optimize data pipeline (AUTOTUNE adjusts buffer size dynamically)
    train_data = train_data.prefetch(AUTOTUNE)
    valid_data = valid_data.prefetch(AUTOTUNE)

    # Train the model
    history = model.fit(
        train_data,
        validation_data=valid_data,
        epochs=epochs,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        validation_steps=valid_data.samples // valid_data.batch_size
    )

    return history


def load_data(data_path, batch_size):
    # Define data augmentation for training images
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Validation data generator with only rescaling
    valid_datagen = ImageDataGenerator(rescale=1.0/255)

    # Load data from directories
    train_data = train_datagen.flow_from_directory(
        data_path + '/train',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )

    valid_data = valid_datagen.flow_from_directory(
        data_path + '/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_data, valid_data


def main(data_path, batch_size, epochs, learning_rate, dropout_rate, unfreeze_layers):
    train_data, valid_data = load_data(data_path, batch_size)
    model = create_model(learning_rate, dropout_rate, unfreeze_layers)
    history = train_model(model, train_data, valid_data, epochs)
    model.save('inception_resnet_model.h5')


if __name__ == "__main__":
    data_path = 'path_to_data'
    batch_size = 32
    epochs = 10
    learning_rate = 0.0001
    dropout_rate = 0.5
    unfreeze_layers = 16
    main(data_path, batch_size, epochs, learning_rate, dropout_rate, unfreeze_layers)